## **基于ResNeXt的树叶分类课程竞赛**

In [1]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l

  Cloning https://github.com/d2l-ai/d2l-zh (to revision release) to /tmp/pip-req-build-gegnu4gb
  Running command git clone -q https://github.com/d2l-ai/d2l-zh /tmp/pip-req-build-gegnu4gb
  Running command git checkout -b release --track origin/release
  Switched to a new branch 'release'
  Branch 'release' set up to track remote branch 'release' from 'origin'.
  Running command git submodule update --init --recursive -q
  Created wheel for d2l: filename=d2l-2.0.0a1-cp37-none-any.whl size=75206 sha256=c89293347fd546ae00a8877baa705e7cfd4df45c823ddbe4d384804b18390f2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gdi9smxn/wheels/91/2a/e4/23538b2879e3cb0f4d4fc58d15bd615432d950f5577c2691f2
Successfully built d2l


In [2]:
!pip install ttach # Test-time Augmentation

In [3]:
!pip install kaggle

In [4]:
import json
token = {"username":"charlesyyun","key":"76d4d71850b9d3d93d3a446d3495664c"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [5]:
!mkdir -p ~/.kaggle

In [6]:
!cp /content/kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle config set -n path -v /content

- path is now set to: /content


In [9]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 6.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=6ec66f45f5a6fcf5d5888383a5a24c905e6f2f0231588b788dfe4106d8e1f541
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [10]:
!kaggle competitions download -c classify-leaves

 94% 185M/197M [00:04<00:00, 28.1MB/s]
100% 197M/197M [00:04<00:00, 41.3MB/s]


In [11]:
!unzip "/content/competitions/classify-leaves/classify-leaves.zip" -d "/content/competitions/classify-leaves"

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/competitions/classify-leaves/images/5500.jpg  
  inflating: /content/competitions/classify-leaves/images/5501.jpg  
  inflating: /content/competitions/classify-leaves/images/5502.jpg  
  inflating: /content/competitions/classify-leaves/images/5503.jpg  
  inflating: /content/competitions/classify-leaves/images/5504.jpg  
  inflating: /content/competitions/classify-leaves/images/5505.jpg  
  inflating: /content/competitions/classify-leaves/images/5506.jpg  
  inflating: /content/competitions/classify-leaves/images/5507.jpg  
  inflating: /content/competitions/classify-leaves/images/5508.jpg  
  inflating: /content/competitions/classify-leaves/images/5509.jpg  
  inflating: /content/competitions/classify-leaves/images/551.jpg  
  inflating: /content/competitions/classify-leaves/images/5510.jpg  
  inflating: /content/competitions/classify-leaves/images/5511.jpg  
  inflating: /content/competitions/classify-leaves/images/5512.jpg  
  inflat

In [12]:
# 导入各种包
import torch
import torch.nn as nn
from torch.nn import functional as F
import ttach as tta
from d2l import torch as d2l

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm


### **整理数据集**

### **数据读取与预处理**

In [13]:
# 看看label文件长啥样
labels_dataframe = pd.read_csv('../content/competitions/classify-leaves/train.csv')
labels_dataframe.head(5)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [14]:
# 把label文件排个序
leaves_labels = sorted(list(set(labels_dataframe['label'])))
n_classes = len(leaves_labels)
print(n_classes)
leaves_labels[:10]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [15]:
# 把label转成对应的数字
class_to_num = dict(zip(leaves_labels, range(n_classes)))
class_to_num

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [16]:
# 再转换回来，方便最后预测的时候使用
num_to_class = {v : k for k, v in class_to_num.items()}
num_to_class

{0: 'abies_concolor',
 1: 'abies_nordmanniana',
 2: 'acer_campestre',
 3: 'acer_ginnala',
 4: 'acer_griseum',
 5: 'acer_negundo',
 6: 'acer_palmatum',
 7: 'acer_pensylvanicum',
 8: 'acer_platanoides',
 9: 'acer_pseudoplatanus',
 10: 'acer_rubrum',
 11: 'acer_saccharinum',
 12: 'acer_saccharum',
 13: 'aesculus_flava',
 14: 'aesculus_glabra',
 15: 'aesculus_hippocastamon',
 16: 'aesculus_pavi',
 17: 'ailanthus_altissima',
 18: 'albizia_julibrissin',
 19: 'amelanchier_arborea',
 20: 'amelanchier_canadensis',
 21: 'amelanchier_laevis',
 22: 'asimina_triloba',
 23: 'betula_alleghaniensis',
 24: 'betula_jacqemontii',
 25: 'betula_lenta',
 26: 'betula_nigra',
 27: 'betula_populifolia',
 28: 'broussonettia_papyrifera',
 29: 'carpinus_betulus',
 30: 'carpinus_caroliniana',
 31: 'carya_cordiformis',
 32: 'carya_glabra',
 33: 'carya_ovata',
 34: 'carya_tomentosa',
 35: 'castanea_dentata',
 36: 'catalpa_bignonioides',
 37: 'catalpa_speciosa',
 38: 'cedrus_atlantica',
 39: 'cedrus_deodara',
 40: 'c

In [17]:
# 继承pytorch的dataset，创建自己的
class TrainValidData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform=None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式
        self.transform = transform

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[1:,1])
        # 计算 length
        self.data_len = len(self.data_info.index) - 1

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)

        # 得到图像的 label
        label = self.label_arr[index]
        number_label = class_to_num[label]

        return (img_as_img, number_label)  #返回每一个index对应的图片数据和对应的label

    def __len__(self):
        return self.data_len

In [18]:
# 继承pytorch的dataset，创建自己的
class TestData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform = None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.transform = transform
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 计算 length
        self.data_len = len(self.data_info.index) - 1
        
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)


        return img_as_img

    def __len__(self):
        return self.data_len

In [19]:
train_transform = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # 添加随机噪声
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [29]:
train_val_path = '../content/competitions/classify-leaves/train.csv'
test_path = '../content/competitions/classify-leaves/test.csv'
# csv文件中已经images的路径了，因此这里只到上一级目录
img_path = '../content/competitions/classify-leaves/'

train_val_dataset = TrainValidData(train_val_path, img_path)
test_dataset = TestData(test_path, img_path, transform = val_test_transform)
print(train_val_dataset.data_info)
print(test_dataset.data_info)

                      0                        1
0                 image                    label
1          images/0.jpg         maclura_pomifera
2          images/1.jpg         maclura_pomifera
3          images/2.jpg         maclura_pomifera
4          images/3.jpg         maclura_pomifera
...                 ...                      ...
18349  images/18348.jpg          aesculus_glabra
18350  images/18349.jpg  liquidambar_styraciflua
18351  images/18350.jpg            cedrus_libani
18352  images/18351.jpg      prunus_pensylvanica
18353  images/18352.jpg          quercus_montana

[18354 rows x 2 columns]
                     0
0                image
1     images/18353.jpg
2     images/18354.jpg
3     images/18355.jpg
4     images/18356.jpg
...                ...
8796  images/27148.jpg
8797  images/27149.jpg
8798  images/27150.jpg
8799  images/27151.jpg
8800  images/27152.jpg

[8801 rows x 1 columns]


### **模型**

In [21]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# resnext50_32x4d模型
def resnext_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.resnext50_32x4d(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [22]:
# 看一下是在cpu还是GPU上
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [23]:
!nvidia-smi

Wed Jun 16 05:19:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
# Configuration options
k_folds = 5
num_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-3
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

### **训练**

In [25]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids,valid_ids) in enumerate(kfold.split(train_val_dataset)):

  # Print
  print(f'FOLD {fold}')
  print('--------------------------------------')

  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

  # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                      TrainValidData(train_val_path, img_path, transform = train_transform), 
                      batch_size=128, sampler=train_subsampler, num_workers=2)
  validloader = torch.utils.data.DataLoader(
                      TrainValidData(train_val_path, img_path, transform = val_test_transform),
                      batch_size=128, sampler=valid_subsampler, num_workers=2)
  
  # Initialize a model and put it on the device specified.
  model = resnext_model(176)
  model = model.to(device)
  model.device = device
  
  # Initialize optimizer
  optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay= weight_decay)
  scheduler = CosineAnnealingLR(optimizer,T_max=10)

  # Run the training loop for defined number of epochs
  for epoch in range(0,num_epochs):
    model.train()
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    # These are used to record information in training
    train_losses = []
    train_accs = []
    # Iterate the training set by batches
    for batch in tqdm(trainloader):
      # Move images and labels to GPU
      imgs, labels = batch
      imgs = imgs.to(device)
      labels = labels.to(device)
      # Forward the data
      logits = model(imgs)
      # Calculate loss
      loss = loss_function(logits,labels)
      # Clear gradients in previous step
      optimizer.zero_grad()
      # Compute gradients for parameters
      loss.backward()
      # Update the parameters with computed gradients
      optimizer.step()
      # Compute the accuracy for current batch.
      acc = (logits.argmax(dim=-1) == labels).float().mean()
      # Record the loss and accuracy.
      train_losses.append(loss.item())
      train_accs.append(acc)
    print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
    scheduler.step()
    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = np.sum(train_losses) / len(train_losses)
    train_acc = np.sum(train_accs) / len(train_accs)
    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

  # Train process (all epochs) is complete
  print('Training process has finished. Saving trained model.')
  print('Starting validation')

  # Saving the model
  print('saving model with acc {:.3f}'.format(train_acc))
  save_path = f'./model-fold-{fold}.pth'
  torch.save(model.state_dict(),save_path)
  # Start Validation
  model.eval()
  valid_losses = []
  valid_accs = []
  with torch.no_grad():
    for batch in tqdm(validloader):
      imgs, labels = batch
      # No gradient in validation
      logits = model(imgs.to(device))
      loss = loss_function(logits,labels.to(device))
      acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
      # Record loss and accuracy
      valid_losses.append(loss.item())        
      valid_accs.append(acc)
    # The average loss and accuracy
    valid_loss = np.sum(valid_losses)/len(valid_losses)
    valid_acc = np.sum(valid_accs)/len(valid_accs)
    print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print('Accuracy for fold %d: %d' % (fold, valid_acc))
    print('--------------------------------------')
    results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} ')
  total_summation += value
print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0/115 [00:00<?, ?it/s]

Starting epoch 1


  0%|          | 0/115 [00:00<?, ?it/s]

...

第29个epoch的学习率：0.000095
[ Train | 029/030 ] loss = 0.01128, acc = 0.99368
Starting epoch 30


  0%|          | 0/29 [00:00<?, ?it/s]

第30个epoch的学习率：0.000024
[ Train | 030/030 ] loss = 0.01063, acc = 0.99447
Training process has finished. Saving trained model.
Starting validation
saving model with acc 0.994


100%|██████████| 29/29 [00:05<00:00,  5.17it/s]

[ Valid | 030/030 ] loss = 0.10856, acc = 0.96364
Accuracy for fold 4: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.9621729850769043 
Fold 1: 0.9601602554321289 
Fold 2: 0.9580360651016235 
Fold 3: 0.958663284778595 
Fold 4: 0.9636377096176147 
Average: 0.9605340361595154 


### **预测**

In [30]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=128, num_workers=2)

In [31]:
## predict
model = resnext_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
  model_path = f'./model-fold-{test_fold}.pth'
  saveFileName = f'./submission-fold-{test_fold}.csv'
  model.load_state_dict(torch.load(model_path))

  # Make sure the model is in eval mode.
  # Some modules like Dropout or BatchNorm affect if the model is in training mode.
  model.eval()
  tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))

  # Initialize a list to store the predictions.
  predictions = []
  # Iterate the testing set by batches.
  for batch in tqdm(testloader):
      
      imgs = batch
      with torch.no_grad():
          logits = tta_model(imgs.to(device))
      
      # Take the class with greatest logit as prediction and record it.
      predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  preds = []
  for i in predictions:
      preds.append(num_to_class[i])

  test_data = pd.read_csv(test_path)
  test_data['label'] = pd.Series(preds)
  submission = pd.concat([test_data['image'], test_data['label']], axis=1)
  submission.to_csv(saveFileName, index=False)
  print("ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

  0%|          | 0/69 [00:00<?, ?it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


  0%|          | 0/69 [00:00<?, ?it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


  0%|          | 0/69 [00:00<?, ?it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


  0%|          | 0/69 [00:00<?, ?it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [00:44<00:00,  1.55it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


### **最终投票Bagging**

In [33]:
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
df2 = pd.read_csv('./submission-fold-2.csv')
df3 = pd.read_csv('./submission-fold-3.csv')
df4 = pd.read_csv('./submission-fold-4.csv')

In [35]:
list_num_label0 = []
for i in df0['label']:
  list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [36]:
list_num_label1 = []
for i in df1['label']:
  list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [37]:
list_num_label2 = []
for i in df2['label']:
  list_num_label2.append(class_to_num[i])
df2['num_label2']=list_num_label2
df2.head()

,image,label,num_label2
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [38]:
list_num_label3 = []
for i in df3['label']:
  list_num_label3.append(class_to_num[i])
df3['num_label3']=list_num_label3
df3.head()

,image,label,num_label3
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [39]:
list_num_label4 = []
for i in df4['label']:
  list_num_label4.append(class_to_num[i])
df4['num_label4']=list_num_label4
df4.head()

,image,label,num_label4
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [40]:
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,22
1,images/18354.jpg,121
2,images/18355.jpg,120
3,images/18356.jpg,102
4,images/18357.jpg,120


In [41]:
df_all['num_label1']=list_num_label1
df_all['num_label2']=list_num_label2
df_all['num_label3']=list_num_label3
df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4
0,images/18353.jpg,22,22,22,22,22
1,images/18354.jpg,121,121,121,121,121
2,images/18355.jpg,120,120,120,120,120
3,images/18356.jpg,102,102,102,102,102
4,images/18357.jpg,120,120,120,120,120


In [43]:
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,8760,8761,8762,8763,8764,8765,8766,8767,8768,8769,8770,8771,8772,8773,8774,8775,8776,8777,8778,8779,8780,8781,8782,8783,8784,8785,8786,8787,8788,8789,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,22,121,120,102,120,74,83,91,165,169,142,145,22,35,105,91,101,143,103,58,65,92,44,133,69,77,10,102,144,52,84,63,32,147,130,127,82,6,100,0,...,53,17,4,17,128,41,4,147,39,61,61,61,41,172,170,128,53,4,117,41,4,159,4,73,53,73,17,17,4,4,117,53,117,53,73,119,53,117,73,73
num_label1,22,121,120,102,120,74,84,91,165,169,142,145,22,35,105,91,132,143,103,58,65,92,44,133,69,77,10,102,144,52,84,63,32,147,130,127,82,6,100,0,...,53,17,4,17,128,41,4,147,39,61,61,61,41,172,172,128,53,4,117,41,4,159,4,73,53,73,17,17,4,4,117,53,117,53,73,119,53,117,73,73
num_label2,22,121,120,102,120,74,84,91,165,169,142,145,22,35,0,91,101,143,103,58,65,92,44,133,69,77,10,102,144,52,84,63,32,147,130,127,82,6,100,0,...,53,17,4,17,128,41,4,147,39,61,61,61,41,172,170,128,53,4,117,41,4,159,4,73,53,73,17,17,4,4,117,53,117,53,73,119,53,117,73,73
num_label3,22,121,120,102,120,74,84,91,165,169,142,145,22,35,155,91,101,143,103,58,65,92,44,133,69,77,10,102,144,52,84,63,32,147,130,127,82,6,100,0,...,53,17,4,17,128,41,4,147,39,61,61,61,41,172,172,128,53,4,117,41,4,159,151,73,53,73,17,17,4,4,117,53,117,53,73,119,53,117,73,73
num_label4,22,121,120,102,120,74,83,91,165,169,142,145,22,35,105,91,101,143,103,58,65,92,44,133,69,77,10,102,144,52,84,63,32,147,130,127,82,6,100,0,...,53,17,4,17,128,41,4,147,39,61,61,61,41,172,172,128,53,4,117,41,4,159,4,73,53,73,17,17,4,4,117,53,117,53,73,110,53,117,73,73


In [46]:
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1,2,3,4
0,22.0,NaN,NaN,NaN,NaN
1,121.0,NaN,NaN,NaN,NaN
2,120.0,NaN,NaN,NaN,NaN
3,102.0,NaN,NaN,NaN,NaN
4,120.0,NaN,NaN,NaN,NaN


In [48]:
voting_class = []
for each in df_mode[0]:
  voting_class.append(num_to_class[each])
voting_class

['asimina_triloba',
 'platanus_occidentalis',
 'platanus_acerifolia',
 'pinus_bungeana',
 'platanus_acerifolia',
 'koelreuteria_paniculata',
 'magnolia_tripetala',
 'metasequoia_glyptostroboides',
 'tilia_cordata',
 'ulmus_americana',
 'quercus_marilandica',
 'quercus_muehlenbergii',
 'asimina_triloba',
 'castanea_dentata',
 'pinus_echinata',
 'metasequoia_glyptostroboides',
 'picea_pungens',
 'quercus_michauxii',
 'pinus_cembra',
 'cryptomeria_japonica',
 'fraxinus_pennsylvanica',
 'morus_alba',
 'cercis_canadensis',
 'ptelea_trifoliata',
 'halesia_tetraptera',
 'liriodendron_tulipifera',
 'acer_rubrum',
 'pinus_bungeana',
 'quercus_montana',
 'cornus_mas',
 'magnolia_tripetala',
 'ficus_carica',
 'carya_glabra',
 'quercus_palustris',
 'prunus_virginiana',
 'prunus_serotina',
 'magnolia_macrophylla',
 'acer_palmatum',
 'picea_orientalis',
 'abies_concolor',
 'quercus_virginiana',
 'liriodendron_tulipifera',
 'stewartia_pseudocamellia',
 'pinus_pungens',
 'gleditsia_triacanthos',
 'ste

In [49]:
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4,label
0,images/18353.jpg,22,22,22,22,22,asimina_triloba
1,images/18354.jpg,121,121,121,121,121,platanus_occidentalis
2,images/18355.jpg,120,120,120,120,120,platanus_acerifolia
3,images/18356.jpg,102,102,102,102,102,pinus_bungeana
4,images/18357.jpg,120,120,120,120,120,platanus_acerifolia


In [51]:
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,platanus_occidentalis
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia


In [52]:
df_submission.to_csv('./submission-voting.csv', index=False)
print('Final voting results successfully saved!')

Final voting results successfully saved!
